In [1]:
import pandas as pd

df_menu = pd.read_excel('./url.xlsx', header=0)
list_url = df_menu['url'].to_list()

print(len(list_url))
list_url

In [ ]:
df = pd.DataFrame(columns=['Part Number',
                           'Category',
                           'OE',
                           'vehicle'
                           'src',
                           'url',
                           'status'])

In [4]:
from selenium.webdriver import ChromeOptions

option = ChromeOptions()
option.add_experimental_option('useAutomationExtension', False)
option.add_argument('blink-settings=imagesEnabled=false')
option.add_argument('--disable-blink-features=AutomationControlled')
option.add_argument('--disable-dev-shm-usage')
option.add_argument('--disable-gpu')
option.add_experimental_option('excludeSwitches', ['enable-automation'])
# option.add_argument('--headless')
option.add_argument('--no-sandbox')
option.add_argument('--proxy-server=http://j933.kdltps.com:15818')
option.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36')
option.page_load_strategy = 'eager'

In [6]:
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.service import Service

browser = Chrome(service=Service('../../chromedriver.exe'), options=option)
browser.maximize_window()

In [7]:
i = 0

browser.get(list_url[i])

In [9]:
from selenium.webdriver.support.wait import WebDriverWait

wait = WebDriverWait(browser, 60, 0.1, None)

In [11]:
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

wait.until(EC.presence_of_element_located((By.XPATH, '//span[@id="lblProductName"]')))
part_number = browser.find_element(By.XPATH, '//span[@id="lblProductName"]').text

part_number

'746-006'

In [12]:
wait.until(EC.presence_of_element_located((By.XPATH, '//span[@id="lblProductDesc"]')))
category = browser.find_element(By.XPATH, '//span[@id="lblProductDesc"]').text

category

'Door Lock Actuator - Left Sliding Door'

In [13]:
wait.until(EC.presence_of_element_located((By.XPATH, '//section[@id="productOE"]/descendant::*/th[@scope="row"]')))
oe = browser.find_element(By.XPATH, '//section[@id="productOE"]/descendant::*/th[@scope="row"]').text

oe

'72663SHJA11'

In [14]:
dict_vehicle = {}
wait.until(EC.presence_of_element_located((By.XPATH, '//iframe[@id="ifDetailApp"]')))
for _ in range(100):
    if browser.find_element(By.XPATH, '//iframe[@id="ifDetailApp"]').get_attribute('src') == '':
        time.sleep(1)
iframe = browser.find_element(By.XPATH, '//iframe[@id="ifDetailApp"]')
browser.switch_to.frame(iframe)

In [15]:
wait.until(EC.presence_of_element_located((By.XPATH, '//tr[@class="detail-app-row"]')))
list_row = browser.find_elements(By.XPATH, '//tr[@class="detail-app-row"]')
for row in list_row:
    make = row.find_element(By.XPATH, './td[3]').text
    model = row.find_element(By.XPATH, './td[2]').text
    year = row.find_element(By.XPATH, './td[1]').text
    
    if make not in dict_vehicle:
        dict_vehicle[make] = {model: [year]}
    else:
        if model not in dict_vehicle[make]:
            dict_vehicle[make][model] = [year]
        else:
            dict_vehicle[make][model].append(year)
                
list_vehicle = []
for make in dict_vehicle:
    for model in dict_vehicle[make]:
        list_vehicle.append(make + ' ' + model + ' ' + min(dict_vehicle[make][model]) + '-' + max(dict_vehicle[make][model]))
        
vehicle = '\n'.join(list_vehicle)
browser.switch_to.parent_frame()

vehicle

'Odyssey Honda 2005-2010'

In [16]:
wait.until(EC.presence_of_element_located((By.XPATH, '//img[@id="imgProductMain"]')))
src = browser.find_element(By.XPATH, '//img[@id="imgProductMain"]').get_attribute('src').replace('/icon/', '/medium/')

src

'https://static.dormanproducts.com/images/product/medium/746-006-007.JPG'

In [ ]:
browser.close()

In [18]:
status = 'ok'
df_temp = pd.DataFrame({'Part Number': part_number,
                        'Category': category,
                        'OE': oe,
                        'vehicle': vehicle,
                        'src': src,
                        'url': list_url[i],
                        'status': ['ok']})

df = pd.concat([df, df_temp], ignore_index=True).fillna('')

df

,Part Number,Category,OE,车型图片链接,链接,status,车型,图片链接
0,746-006,Door Lock Actuator - Left Sliding Door,72663SHJA11,,https://www.dormanproducts.com/p-137759-746-00...,ok,Odyssey Honda 2005-2010,https://static.dormanproducts.com/images/produ...
